Import Libs

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# own packages
from preprocess_data import clean_text
from preprocess_data import lemmatize_text
from preprocess_data import tfidf_vec_transform, tfidf_vec_fit_transform
from preprocess_data import count_vectorizer
from models import model_sfnn_train
from models import model_rf_train, model_naives_bayes, model_logistic_regression, model_multinominalNB_train, predict_values, model_xgboost



Function for reading the data

In [2]:
# get csv data
def read_Data(path):

    # Define column names
    column_names = ['label', 'text']

    df = pd.read_csv(path, delimiter='\t', encoding="utf-8-sig", header=None, names=column_names)
    df.columns = df.columns.str.replace('\ufeff', '')
    df['label'] = df['label'].astype(str).str.replace('\ufeff', '')
    df['label'] = df['label'].astype(int)
    
    return df



In [3]:
## read data windows
data_train_val = read_Data(r".\data\training_data_lowercase.csv")           # used for train and val data
data_test = read_Data(r".\data\testing_data_lowercase_nolabels.csv")        # data for the predictions

## read data Mac
#data_train_val = read_Data("data/training_data_lowercase.csv")           # used for train and val data
#data_test = read_Data("data/testing_data_lowercase_nolabels.csv")        # data for the predictions


## split data_train into train and val data
data_train, data_val = train_test_split(data_train_val, test_size=0.08, random_state=42, stratify=data_train_val['label'])


In [ ]:
## preview data
#print(data_train.head, "\n")
#print(data_test.head, "\n")
print("Shape train data:\n", data_train_val.shape)
print("Shape test data:\n", data_test.shape)

# clean data
data_train['cleaned_text'] = data_train['text'].apply(clean_text)
data_val['cleaned_text'] = data_val['text'].apply(clean_text)
data_test['cleaned_text'] = data_test['text'].apply(clean_text)

print("Cleand text train: \n", data_train["cleaned_text"].head, "\n")
print("Cleand text test: \n", data_test["cleaned_text"].head, "\n")


## lemmatize data
data_train['lemmatized_text'] = data_train['cleaned_text'].apply(lemmatize_text)
data_val['lemmatized_text'] = data_val['cleaned_text'].apply(lemmatize_text)
data_test['lemmatized_text'] = data_test['cleaned_text'].apply(lemmatize_text)

print("lemmatized_text train: \n", data_train['lemmatized_text'].head, "\n")
print("lemmatized_text test: \n", data_test['lemmatized_text'].head, "\n")

# calc tf-idf matrix
vectorizer = TfidfVectorizer(ngram_range=(1,2))

tfidf_matrix_train = vectorizer.fit_transform(data_train['lemmatized_text'])
tfidf_matrix_val = vectorizer.transform(data_val['lemmatized_text'])
tfidf_matrix_test = vectorizer.transform(data_test['lemmatized_text'])

# calc tf-idf matrix with uncleand data
vectorizer2 = TfidfVectorizer(stop_words="english", ngram_range=(1, 3))
X_train_tfidf = vectorizer2.fit_transform(data_train['text'])
X_val_tfidf = vectorizer2.transform(data_val['text'])
X_test_tfidf = vectorizer2.transform(data_test['text'])

#print("tfidf_matrix_train_val: \n", tfidf_matrix_train, "\n")
#print("tfidf_matrix_test: \n", tfidf_matrix_test, "\n")


# count vectorizer
#X_train_vec = count_vectorizer(data_train['lemmatized_text']).toarray()
#X_val_vec = count_vectorizer(data_val['lemmatized_text']).toarray()



Shape train data:
 (34152, 2)
Shape test data:
 (9984, 2)
Cleand text train: 
 <bound method NDFrame.head of 6602     [fox, host, throws, hissy, fit, cant, call, pr...
930      [trump, unpopular, crowds, bussed, cheer, pola...
29744    [congratulations, eu, moves, brexit, phase, tw...
24734    [trumps, national, security, adviser, vows, ta...
15772    [republican, turns, tables, fbi, deputy, direc...
                               ...                        
27469    [us, pledges, strong, response, event, another...
27793    [republican, presidential, candidate, cruz, ra...
883                                         [like, father]
25547    [congress, passes, funding, bill, averting, go...
3939     [hilariously, amazing, video, weve, waiting, e...
Name: cleaned_text, Length: 31419, dtype: object> 

Cleand text test: 
 <bound method NDFrame.head of 0       [copycat, muslim, terrorist, arrested, assault...
1       [wow, chicago, protester, caught, camera, admi...
2       [germanys, fdp, 

In [5]:
# Naive Bayes Model
model_nb = model_naives_bayes(tfidf_matrix_train, data_train['label'], tfidf_matrix_val, data_val['label'])


Train Accuracy:1.00

Validation Accuracy: 0.94

classification_report:
               precision    recall  f1-score   support

           0     0.9371    0.9538    0.9454      1406
           1     0.9501    0.9322    0.9410      1327

    accuracy                         0.9433      2733
   macro avg     0.9436    0.9430    0.9432      2733
weighted avg     0.9434    0.9433    0.9433      2733



In [6]:
# XGBoost
model_xgboost(tfidf_matrix_train, data_train['label'], tfidf_matrix_val, data_val['label'])

Train Accuracy: 0.90

Validation Accuracy: 0.83

Classification Report:
               precision    recall  f1-score   support

           0     0.8213    0.8464    0.8336      1406
           1     0.8318    0.8048    0.8181      1327

    accuracy                         0.8262      2733
   macro avg     0.8265    0.8256    0.8259      2733
weighted avg     0.8264    0.8262    0.8261      2733



In [7]:
# Random Forest
model_rf = model_rf_train(tfidf_matrix_train, tfidf_matrix_val, data_train['label'], data_val['label'])

Train Accuracy: 100.00%
Random forest model
accuracy: 0.9008415660446396
Classification report:
               precision    recall  f1-score   support

           0       0.89      0.92      0.90      1406
           1       0.91      0.89      0.90      1327

    accuracy                           0.90      2733
   macro avg       0.90      0.90      0.90      2733
weighted avg       0.90      0.90      0.90      2733



In [8]:
# Multinomial Naive Bayes (MultinomialNB) classifier
#model_mnb = model_multinominalNB_train(data_train['lemmatized_text'], data_val['lemmatized_text'], data_train['label'], data_val['label'])


In [9]:
# Simple Feedforward NN -> no good results
model_sfnn_train(tfidf_matrix_train, data_train['label'], tfidf_matrix_val, data_val['label'])


d:\ownCloud\Dokumente\Ironhack\.venv\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8485 - loss: 0.3338 - val_accuracy: 0.9312 - val_loss: 0.1840
Epoch 2/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9369 - loss: 0.1599 - val_accuracy: 0.9323 - val_loss: 0.1901
Epoch 3/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9512 - loss: 0.1241 - val_accuracy: 0.9301 - val_loss: 0.1935
Epoch 4/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9653 - loss: 0.0940 - val_accuracy: 0.9385 - val_loss: 0.1953
Epoch 5/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9770 - loss: 0.0674 - val_accuracy: 0.9363 - val_loss: 0.2255
Epoch 6/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9847 - loss: 0.0467 - val_accuracy: 0.9356 - val_loss: 0.2562
982/982 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Train Accuracy: 94.65%
Validation Accuracy: 93.12%
Validation Classification Report:
               precision    recall  f1-score   support

           

<Sequential name=sequential, built=True>

In [10]:
model_lr = model_logistic_regression(tfidf_matrix_train, data_train['label'], tfidf_matrix_val, data_val['label'])


Train Accuracy: 97.42%

Validation Accuracy: 92.86%

Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.92      0.93      1406
           1       0.92      0.94      0.93      1327

    accuracy                           0.93      2733
   macro avg       0.93      0.93      0.93      2733
weighted avg       0.93      0.93      0.93      2733



In [11]:
## make predictions on test data
filepath = r".\data\testing_data_lowercase_labels.csv"

#predict_values(model_rf, tfidf_matrix_test, data_test, filepath)
predict_values(model_nb, tfidf_matrix_test, data_test, filepath)

✅ Predictions saved to: .\data\testing_data_lowercase_labels.csv
